<a href="https://colab.research.google.com/github/Chanito/mlbourbaki/blob/main/LDA_con_sklearn20_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis  #biblioteca que extrae información de un modelo LDA para obtener una visualización interactiva

     |████████████████████████████████| 1.7MB 5.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 9.9MB 18.0MB/s 
     |████████████████████████████████| 15.3MB 150kB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-cp37-none-any.whl size=136897 sha256=1e4bccd2cf2d578e95f979651bfa3281c8f7971b7f4674812a88721b339edbb1
  Stored in directory: /root/.cache/pip/wheels/a0/9c/fc/c6e00689d35c82cf96a8adc70edfe7ba7904374fdac3240ac2
Successfully built pyLDAvis
ERROR: tensorflow 2.5.0 has requirement numpy~=1.19.2, but you'll have numpy 1.20.3 which is incompatible.
ERROR: google-colab 1.0.0 has requirement pandas~=1.1.0; python_version >= "3.0", but you'll have pandas 1.2.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0

In [1]:
import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer

import pyLDAvis
import matplotlib.pyplot as plt 
import seaborn as sns  

nltk.download('stopwords')
nltk.download('wordnet')  #WordNetLemmatizer
nltk.download('punkt')


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).ep

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

En colab, al instalar la librería pyLDAvis causa un conflicto con Pandas. Si esto se refleja al correr la siguiente linea es necesario reiniciar el entorno de ejecución.

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Datos/Headlines/abcnews-date-text.csv', parse_dates=['publish_date'] )
print(df.shape)
df.head()

(1226258, 2)


,publish_date,headline_text
0,2003-02-19,aba decides against community broadcasting lic...
1,2003-02-19,act fire witnesses must be aware of defamation
2,2003-02-19,a g calls for infrastructure protection summit
3,2003-02-19,air nz staff in aust strike for pay rise
4,2003-02-19,air nz strike to affect australian travellers


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df.publish_date.min(), df.publish_date.max()

(Timestamp('2003-02-19 00:00:00'), Timestamp('2020-12-31 00:00:00'))

In [6]:
# promedio de noticias por día
len(df)/(df.publish_date.max()-df.publish_date.min()).days


187.93226053639847

In [7]:
df.sample(25)

,publish_date,headline_text
484099,2009-08-25,storms bring relief to parched gippsland
590481,2011-01-27,fears apy school attendance rates falling
537235,2010-05-12,extended interview brendan smyth speaks with 666
1045466,2016-08-31,hunter paedophile priest was like jekyll and hyde
1146123,2018-11-02,premiers spelling bee 2018 sydney
861662,2014-03-20,rural sa challenging season 2003
361464,2008-02-05,axe to fall on mitsubishi in march
1019621,2016-04-06,queensland urged to reconsider daylight saving
25590,2003-06-22,abortion ship denied entry to polish port
1102045,2017-10-11,sydney city council to open up late night shop...


In [9]:
def preprocesar(texto):
  #convierte a minúsculas
  texto = texto.lower()

  #elimina stopwords
  stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
  texto = stop.sub('', texto) 

  #lematizar y quedarnos con palabras que tengan más de tres caracteres
  lemmatizer = WordNetLemmatizer()
  texto = texto.split()
  texto = ' '.join([lemmatizer.lemmatize(i) for i in texto if len(i)>3])
  
  return(texto)

In [10]:
%%time
df['headline_pp']=df['headline_text'].apply(preprocesar)
df.head()

CPU times: user 3min 7s, sys: 15 s, total: 3min 22s
Wall time: 3min 22s


In [11]:
#vectorizer = TfidfVectorizer()           # 92403 tamaño de vocabulario
vectorizer = TfidfVectorizer(min_df=3)    #42695
vectorizer = TfidfVectorizer(min_df=20, ngram_range=(1,2))
BOW = vectorizer.fit_transform(df['headline_pp'])
BOW.shape

(1226258, 36660)

In [12]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=20,learning_method='online',random_state=42,max_iter=10) 

In [13]:
%%time
lda_top=lda_model.fit_transform(BOW)

CPU times: user 1h 28min 7s, sys: 51.2 s, total: 1h 28min 59s
Wall time: 1h 28min 53s


In [14]:
import pickle
# do we need to save lda_model instead?
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/LDA20_10iter.pikle', 'wb') as f:
  pickle.dump(lda_top, f)

In [26]:
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/vectorizer.pikle', 'wb') as f:
  pickle.dump(vectorizer, f)

In [31]:
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/BOW.pikle', 'wb') as f:
  pickle.dump(BOW, f)

In [27]:
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/vectorizer.pikle', 'rb') as f:
  vectorizer1 = pickle.load(f)

In [32]:
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/BOW.pikle', 'rb') as f:
  BOW1 = pickle.load(f)

In [33]:
with open('/content/drive/MyDrive/Modelos/modelosLDA/LDA_sklearn/LDA20_10iter.pikle', 'rb') as f:
  lda_top1 = pickle.load(f)

In [17]:
%%time
lda_model.perplexity(BOW, sub_sampling = False)

CPU times: user 1min 43s, sys: 952 ms, total: 1min 44s
Wall time: 1min 43s


82111.13575853143

In [18]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_model, BOW, vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
12    -0.181567 -0.052764       1        1  5.621199
9      0.096530  0.076398       2        1  5.404073
3      0.113550 -0.055457       3        1  5.173005
10    -0.066508 -0.094020       4        1  5.132125
15     0.118551  0.168088       5        1  5.050068
2      0.111031  0.130990       6        1  5.049910
19    -0.107039 -0.168078       7        1  5.038129
16     0.158912 -0.154940       8        1  5.028129
0      0.114011  0.082870       9        1  5.017374
18    -0.174603  0.104250      10        1  4.961355
13     0.032126  0.079531      11        1  4.947165
14    -0.119641  0.148698      12        1  4.946355
7     -0.133201  0.007058      13        1  4.933431
8     -0.016131 -0.005134      14        1  4.924098
6      0.065865 -0.150169      15        1  4.910854
4     -0.125871  0.010796      16        1  4.904815
5      0.021464 -0.045856      17        1  4.803747
1      0.157828  0.012736      18        1  4.741924
17    -0.073024  0.110813      19        1  4.725842
11     0.007715 -0.205809      20        1  4.686401, topic_info=              Term         Freq        Total Category  logprob  loglift
2060     australia  7515.000000  7515.000000  Default  30.0000  30.0000
7467   coronavirus  6641.000000  6641.000000  Default  29.0000  29.0000
2166    australian  5839.000000  5839.000000  Default  28.0000  28.0000
35982        woman  4868.000000  4868.000000  Default  27.0000  27.0000
17174    interview  4743.000000  4743.000000  Default  26.0000  26.0000
...            ...          ...          ...      ...      ...      ...
25526        probe  1153.255197  1346.888065  Topic20  -4.7284   2.9053
3809         boost  1260.286736  1593.457671  Topic20  -4.6397   2.8259
24347         plan  1859.189769  3523.767421  Topic20  -4.2509   2.4211
16924      injured   827.436399  1256.478960  Topic20  -5.0604   2.6428
24622       police   871.334503  7387.554815  Topic20  -5.0087   0.9230

[714 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
113        3  0.998623     2014
116        2  0.999427     2015
118       11  0.998888     2016
122       17  0.998508     2018
126       11  0.998836     2020
...      ...       ...      ...
36485      4  0.225028     year
36485     17  0.774727     year
36562     13  0.999517    young
36585      7  0.998796    youth
36621      5  0.998929  zealand

[798 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 10, 4, 11, 16, 3, 20, 17, 1, 19, 14, 15, 8, 9, 7, 5, 6, 2, 18, 12])